<a href="https://colab.research.google.com/github/EtzionR/NLP4GeoAI/blob/main/Text_to_Geo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
%%bash
rm -rf NLP4GeoAI
git clone https://github.com/EtzionR/NLP4GeoAI.git

Cloning into 'NLP4GeoAI'...


In [3]:
df = pd.read_csv('NLP4GeoAI/data.csv')

print(f'Dataframe shape: {df.shape}')

df.head()

Dataframe shape: (23072, 2)


,Text,Source
0,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ...",ontonotes5
1,Rules that set standards for products or gover...,ontonotes5
2,Determining when handicapped access is require...,ontonotes5
3,"``It's very costly and time-consuming ,'' says...",ontonotes5
4,"Next to medical insurance, ``costs of complian...",ontonotes5


In [28]:
from transformers import pipeline

ner = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)

output = ner("Sam Altman visited OpenAI in San Francisco.")

pd.DataFrame(output)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


,entity_group,score,word,start,end
0,PER,0.999172,Sam Altman,0,10
1,ORG,0.997174,OpenAI,19,25
2,LOC,0.999392,San Francisco,29,42


In [8]:
from tqdm import tqdm

outputs = []

for text, source in tqdm(df.values):
    entites = ner(text)

    for entity in entites:

        entity['source'] = source
        entity['text'] = text

        outputs.append(entity)

outputs = pd.DataFrame(outputs)

print(f'NER output shape: {outputs.shape}')

outputs

100%|██████████| 23072/23072 [05:11<00:00, 74.04it/s] 


NER output shape: (69932, 7)


,entity_group,score,word,start,end,source,text
0,PER,0.999557,Malcolm Wallop,16,30,ontonotes5,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ..."
1,ORG,0.996441,LRB,32,35,ontonotes5,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ..."
2,LOC,0.879631,R,37,38,ontonotes5,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ..."
3,LOC,0.794557,Wyo,41,44,ontonotes5,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ..."
4,ORG,0.997694,RRB,47,50,ontonotes5,"Last week, Sen. Malcolm Wallop -LRB- R., Wyo. ..."
...,...,...,...,...,...,...,...
69927,LOC,0.999821,England,158,165,conll2003,He guided Ireland to two successive World Cup ...
69928,ORG,0.999167,Leeds United,17,29,conll2003,The lanky former Leeds United defender did not...
69929,LOC,0.999798,England,56,63,conll2003,The lanky former Leeds United defender did not...
69930,MISC,0.996952,World Cup,150,159,conll2003,The lanky former Leeds United defender did not...


In [18]:
k = 20

top_k_locations = outputs[outputs.entity_group=='LOC'].word.value_counts().head(k).reset_index()
top_k_locations

,word,count
0,China,1484
1,U. S.,1289
2,Taiwan,1252
3,Iraq,605
4,Japan,595
5,US,508
6,Israel,446
7,United States,418
8,New York,406
9,Hong Kong,403


In [12]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="my_geo_app")

location = geolocator.geocode("Tel Aviv, Israel")

print(location.address)
print(location.latitude, location.longitude)

תל־אביב–יפו, נפת תל אביב, מחוז תל אביב, ישראל
32.0852997 34.7818064


In [19]:

from time import sleep as wait

x_coords = []
y_coords = []

for placename in tqdm(top_k_locations.word):
    loc = geolocator.geocode(placename)
    x_coords.append(loc.latitude)
    y_coords.append(loc.longitude)
    wait(1.1)

top_k_locations['x'] = x_coords
top_k_locations['y'] = y_coords

top_k_locations

100%|██████████| 20/20 [00:27<00:00,  1.40s/it]


,word,count,x,y
0,China,1484,35.000074,104.999927
1,U. S.,1289,39.783730,-100.445882
2,Taiwan,1252,23.973937,120.982018
3,Iraq,605,33.095579,44.174977
4,Japan,595,36.574844,139.239418
5,US,508,39.783730,-100.445882
6,Israel,446,30.812425,34.859476
7,United States,418,39.783730,-100.445882
8,New York,406,40.712728,-74.006015
9,Hong Kong,403,22.350627,114.184916


In [30]:
import folium

# Create a map centered on Tel Aviv
m = folium.Map(location=[32.0853, 34.7818], zoom_start=3)

for name,x,y in zip(top_k_locations.word, top_k_locations.x, top_k_locations.y):
    folium.Marker([x,y], popup=name, tooltip=name).add_to(m)

# Display in notebook
m

In [29]:
outputs[outputs.entity_group=='PER'].word.value_counts().head(k).reset_index()


help(folium.Marker)

Help on class Marker in module folium.map:

class Marker(branca.element.MacroElement)
 |  Marker(location: Optional[Sequence[float]] = None, popup: Union[ForwardRef('Popup'), str, NoneType] = None, tooltip: Union[ForwardRef('Tooltip'), str, NoneType] = None, icon: Union[folium.map.Icon, ForwardRef('CustomIcon'), ForwardRef('DivIcon'), NoneType] = None, draggable: bool = False, **kwargs: Union[str, float, bool, Sequence, dict, NoneType])
 |
 |  Create a simple stock Leaflet marker on the map, with optional
 |  popup text or Vincent visualization.
 |
 |  Parameters
 |  ----------
 |  location: tuple or list
 |      Latitude and Longitude of Marker (Northing, Easting)
 |  popup: string or folium.Popup, default None
 |      Label for the Marker; either an escaped HTML string to initialize
 |      folium.Popup or a folium.Popup instance.
 |  tooltip: str or folium.Tooltip, default None
 |      Display a text when hovering over the object.
 |  icon: Icon, CustomIcon or DivIcon, optional
 |  